# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
filepath = os.path.join('..', 'output_data', 'city_weather_records.csv')
df = pd.read_csv(filepath)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Scarborough,54.2797,-0.4044,288.45,88,97,3.01,GB,1627270744
1,Fortuna,40.5982,-124.1573,286.44,92,90,3.09,US,1627270744
2,Livingston,55.9029,-3.5226,289.38,93,90,1.03,GB,1627270744
3,Fort-Shevchenko,44.5086,50.2630,302.44,47,2,7.82,KZ,1627270744
4,Pemba,-12.9740,40.5178,291.54,88,20,3.60,MZ,1627270606


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = df[['Lat', 'Lng']]
humidity = df['Humidity'].astype(float)

In [33]:
# plot heatmap
fig = gmaps.figure()

# create heat layer
heat_layer = gmaps.heatmap_layer(
                                 locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5
                                )
# add layer
fig.add_layer(heat_layer)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# check missing values of the table
df.isnull().sum()

City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       7
Date          0
dtype: int64

In [11]:
df.shape

(737, 9)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df = df.dropna().reset_index()

In [31]:
hotel_df['Hotel Name'] = ''
hotel_df.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,Scarborough,54.2797,-0.4044,288.45,88,97,3.01,GB,1627270744,
1,1,Fortuna,40.5982,-124.1573,286.44,92,90,3.09,US,1627270744,
2,2,Livingston,55.9029,-3.5226,289.38,93,90,1.03,GB,1627270744,
3,3,Fort-Shevchenko,44.5086,50.2630,302.44,47,2,7.82,KZ,1627270744,
4,4,Pemba,-12.9740,40.5178,291.54,88,20,3.60,MZ,1627270606,


In [36]:
# find the closest hotel of each city to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get coordinate from df
    lat = row['Lat']
    lng = row['Lng']
    # add keyword to params dict
    params['location'] = f'{lat},{lng}'

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:      
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [37]:
hotel_df.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,Scarborough,54.2797,-0.4044,288.45,88,97,3.01,GB,1627270744,The Grand Hotel Scarborough
1,1,Fortuna,40.5982,-124.1573,286.44,92,90,3.09,US,1627270744,THE REDWOOD RIVERWALK HOTEL
2,2,Livingston,55.9029,-3.5226,289.38,93,90,1.03,GB,1627270744,Macdonald Houstoun House
3,3,Fort-Shevchenko,44.5086,50.2630,302.44,47,2,7.82,KZ,1627270744,Chagala Bautino Hotel
4,4,Pemba,-12.9740,40.5178,291.54,88,20,3.60,MZ,1627270606,Avani Pemba Beach Hotel


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))